In [1]:
import random
from random import shuffle

from PIL import Image, ImageFilter
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle
import time
import error_table

from sampling_utils import *
from gen_utils import *
from evaluation_utils import *
from config_db import ConfigDB

In [1]:
num_pics = 10
min_num_cars = 1
max_num_cars = 4

domains = [[52,87],[1,37],[0,1],[0.35,1],[0.5,1.5],[0.25,1],[0.5,1],[0.5,1]]
#domains = [[74,77],[1,37],[0,1],[0.4,1],[0.5,0.6],[0.5,0.6],[0.5,0.6],[0.5,0.6]]
types = ['int','int','float','float','float','float','float','float']

gen_misclass = True
store_config_db = False
store_err_tab = False

IOMIN_THRESH = 0.4
IOU_THRESH = 0.5
AP_THRESH = 0.75
REC_THRESH = 0.75
NN_PROB_THRESH = 0.5

if gen_misclass:
    checkpoint = '/home/tommaso/analyzeNN/data/train_0/checkpoint/train/model.ckpt-5000'
    import squeezedet as nn
    net = nn.init(checkpoint)

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
for j in range(min_num_cars,max_num_cars):
    
    total_attempts = 0
    rejections = 0
    
    # Init data structure
    if store_err_tab:
        err_table = error_table()
    if store_config_db:
        config_db = ConfigDB()
    
    path_data_set = './data/tmp/'
    #path_data_set = '/home/tommaso/analyzeNN/data/train_0/mis/mis_' + str(j) + '/'
    
    i = 0
    v = 0

    while i < num_pics:
        
        v = v+1

        # Sample configuration
        num_cars = random.randint(min_num_cars, max_num_cars)
        sample_dims = [1,num_cars,num_cars,num_cars,1,1,1,1]
        sample, norm_sample = get_sample(v, 'random', types=types, domains=domains, dims=sample_dims)
        
        car_models = []
        for c in range(num_cars):
            car_models += [random.choice([2,27,26,28,29,30])]
        sample[1] = car_models
        
        sample[3].sort(reverse=True)
        norm_sample[3].sort(reverse=True)
        
        total_attempts += 1
        
        good_sample = True
        # Check sample distance from stored ones
        if store_config_db:
            sample_pad = pad_sample(sample)
            good_sample = config_db.eps_dist(sample_pad,0.25)
            
            
        if good_sample:
            # Generate image from sample
            img, ground_boxes, img_info = gen_image(sample)

            # Is it a good picture? (cars don't overlap too much)
            save_img = True
            for m in range(len(ground_boxes)):
                for n in range(m+1, len(ground_boxes)):
                    if iomin(ground_boxes[m], ground_boxes[n]) > IOMIN_THRESH:
                        save_img = False

            ap = 0
            recall = 0
                        
            # Check misclassification
            if gen_misclass and save_img:
                # Save tmp image    
                tmp_img_file_name = './tmp.png'
                img.save(tmp_img_file_name)

                # Classify img
                (boxes,probs,cats) = nn.classify(tmp_img_file_name, net, NN_PROB_THRESH) 

                # Not cars
                if any(cats):
                    save_img = True
                else:
                    ap, recall = average_precision(ground_boxes, boxes, IOU_THRESH)
                    save_img =  (ap <= AP_THRESH or recall <= REC_THRESH)                        
                        

            if save_img:
                
                if store_config_db:
                    config_db.add(sample_pad)            
                
                file_name = 'm_h_' + str(j) + '_' + str(i).zfill(6)
                #file_name = 'm_d_' + str(i).zfill(6)
                
                print(file_name + ' ' + str(ap) + ' ' + str(recall))
                
                # Save data
                save_image(img, file_name, path_data_set)
                save_label(ground_boxes, file_name, path_data_set)
                
                # Save in image set
                with open(path_data_set + 'image_set_m_1.txt', "a+") as im_set:
                    im_set.write(file_name + '\n')
                    im_set.close()
                
                if store_err_tab:
                    err_table.update_with_elem(img_info)
                    
                i+=1
        else:
            rejections += 1

#         if(total_attempts %50 == 0):
#             print('gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))
            
print('END gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))

if store_err_tab:
    err_table_file = open(path_data_set + 'error_table' + str(),'w')
    pickle.dump(err_table,err_table_file)
    err_table_file.close()

m_h_1_000000 0.0 0.0
m_h_1_000001 0.5 1.0
m_h_1_000002 0.666666666667 1.0
m_h_1_000003 0.5 0.5
m_h_1_000004 0.75 1.0
m_h_1_000005 0.5 0.5
m_h_1_000006 0.666666666667 1.0
m_h_1_000007 0.0 0.0
m_h_1_000008 0.0 0.0
m_h_1_000009 0.666666666667 1.0
m_h_2_000000 0.666666666667 1.0
m_h_2_000001 0.666666666667 1.0
m_h_2_000002 0.666666666667 0.666666666667
m_h_2_000003 0.5 0.5
m_h_2_000004 0.0 0.0
m_h_2_000005 0.5 0.5
m_h_2_000006 0.666666666667 1.0
m_h_2_000007 0.0 0.0
m_h_2_000008 0.0 0.0
m_h_2_000009 0.75 1.0
m_h_3_000000 0.666666666667 1.0
m_h_3_000001 1.0 0.5
m_h_3_000002 0.5 0.5
m_h_3_000003 0.5 0.5
m_h_3_000004 0.5 1.0
m_h_3_000005 0.666666666667 1.0
m_h_3_000006 0.666666666667 0.666666666667
m_h_3_000007 0.666666666667 1.0
m_h_3_000008 0.666666666667 1.0
m_h_3_000009 0.666666666667 1.0
END gen pics: 901 | misclass pics: 10 | rejections: 0


In [52]:
(boxes,probs,cats) = nn.classify('./data/train_0/mis/images/m_0_3_000049.png', net, NN_PROB_THRESH) 

In [53]:
labels = read_label('./data/train_0/mis/labels/m_0_3_000049.txt')
label_boxes = [ kitti_2_box_format(l) for l in labels ]
ap, recall = average_precision(label_boxes, boxes, IOU_THRESH)

In [54]:
ap

0.75

In [55]:
recall

1.0

In [38]:
boxes

[array([ 584.81933594,  220.34779358,  129.04016113,  105.63809204], dtype=float32),
 array([ 589.07788086,  173.05867004,   62.38708496,   49.52755737], dtype=float32),
 array([ 635.69897461,  169.13327026,   56.59399414,   41.10015869], dtype=float32)]

In [42]:
gb

[[610.0, 145.0, 654.0, 177.0],
 [556.0, 144.0, 612.0, 192.0],
 [518.0, 179.0, 646.0, 265.0]]

In [56]:
net.close()

AttributeError: 'tuple' object has no attribute 'close'

In [5]:
e = pickle.load( open( "error_table", "rb" ) )

In [6]:
type(e)

instance

In [13]:
e.entries.keys()

['foreground0_width',
 'color_sample',
 'road_path',
 'environment',
 'road_id',
 'foreground0car_color',
 'foreground0car_orientation',
 'foreground0_x',
 'foreground0car_category',
 'road_type',
 'foreground0car_id',
 'foreground0type',
 'sharpness_sample',
 'contrast_sample',
 'foreground0car_path',
 'foreground0_height',
 'background_color',
 'foreground0_y',
 'brightness_sample']

In [3]:
num_cars=1
sample_dims = [1,num_cars,num_cars,num_cars,1,1,1,1]
sample, norm_sample = get_sample(0, 'halton', types=types, domains=domains, dims=sample_dims)

IndexError: list index out of range

In [ ]:
MAX_ITERS = 3333
i = 0
j = 0
v = 0
while j < MAX_ITERS:
    
    total_attempts = 0
    rejections = 0
    
    # Init data structure
    if store_err_tab:
        err_table = error_table()
    if store_config_db:
        config_db = ConfigDB()
    
    path_data_set = './data/tmp/'
    #path_data_set = '/home/tommaso/analyzeNN/data/train_0/mis/mis_' + str(j) + '/'

        
    v = v+1

    # Sample configuration
    #num_cars = random.randint(min_num_cars, max_num_cars)
    
    t = time.time()
    
    num_cars = 3
    
    sample_dims = [1,num_cars,num_cars,num_cars,1,1,1,1]
    sample, norm_sample = get_sample(int(random.random()*9999), 'halton', types=types, domains=domains, dims=sample_dims)

    
    #sample[0] = [random.choice([74,66,75])]
    
    car_models = []
    for c in range(num_cars):
        car_models += [random.choice(range(1,37))]
    sample[1] = car_models
    

    sample[3].sort(reverse=True)
    norm_sample[3].sort(reverse=True)

    total_attempts += 1

    good_sample = True
    # Check sample distance from stored ones
    if store_config_db:
        sample_pad = pad_sample(sample)
        good_sample = config_db.eps_dist(sample_pad,0.25)


    if good_sample:
        # Generate image from sample
        img, ground_boxes, img_info = gen_image(sample)

        # Is it a good picture? (cars don't overlap too much)
        save_img = True
        for m in range(len(ground_boxes)):
            for n in range(m+1, len(ground_boxes)):
                if iomin(ground_boxes[m], ground_boxes[n]) > IOMIN_THRESH:
                    save_img = False

        ap = 0
        recall = 0

        # Check misclassification
        if gen_misclass and save_img:
            # Save tmp image    
            tmp_img_file_name = './tmp.png'
            img.save(tmp_img_file_name)
            
            j = j + 1
            print str(j), str(i)

            # Classify img
            (boxes,probs,cats) = nn.classify(tmp_img_file_name, net, NN_PROB_THRESH) 

            # Not cars
            if any(cats):
                save_img = True
            else:
                ap, recall = average_precision(ground_boxes, boxes, IOU_THRESH)
                save_img =  (ap <= AP_THRESH or recall <= REC_THRESH)                        

                
        if save_img:
            
            elapsed = time.time() - t
            
            print elapsed

            if store_config_db:
                config_db.add(sample_pad)            

            file_name = 'm_h_' + str(i).zfill(6)
            #file_name = 'm_d_' + str(i).zfill(6)

            print(file_name + ' ' + str(ap) + ' ' + str(recall))

            # Save data
            save_image(img, file_name, path_data_set)
            save_label(ground_boxes, file_name, path_data_set)

            # Save in image set
            with open(path_data_set + 'image_set_m_1.txt', "a+") as im_set:
                im_set.write(file_name + '\n')
                im_set.close()

            if store_err_tab:
                err_table.update_with_elem(img_info)

            i+=1
    else:
        rejections += 1

#         if(total_attempts %50 == 0):
#             print('gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))

print i

1 0


In [14]:
range(1,3)

[1, 2]

In [15]:
10000/3

3333

In [5]:

# do stuff


In [13]:
3333*7/50

466